In [ ]:
import re
def get_url_input( mode):
        while True:
            fmode = mode.replace('_',' ')
            url = input(f'Enter {fmode} URL : ')
            if mode == 'single_product':
                if ('product' not in url and not re.search(r'https://www\.digikala\.com/product/dkp-\d+/$', url) ):
                    print('URL is not belong to any product check it and Please try again.')
                    continue
            elif mode == 'single_seller':
                pattern = re.compile(r'https://www\.digikala\.com/seller/[A-Z0-9]+/$')
                if  (not pattern.match(url)):
                     print("URL is not belong to any seller check it and Please try again.")
                     continue
            elif mode == 'category':
                pattern = re.compile(r'search/\?q=|/category-|/search/')
                if not ('search/?q=' in url or '/category-' in url or '/search/' in url):
                    print('Wrong category URL, try one more time')
                    continue
            break
        return url
get_url_input( 'single_product')

In [ ]:
def run_scraper_category(self):
        while True :
            category_url = input("enter category url to crawl: ")

            if 'search/?q=' in category_url or '/category-' in category_url or '/search/' in category_url:
                break
            else:
                self.log.info('wrong category url, try one more time')       
        scroll_count = input("Please enter the number of page scroll rates (Example 5 ) : ")
        self.get_driver()
        try:
            scroll_count = int(scroll_count)
        except ValueError:
            self.log.error("The number of scrolling times must be a number. By default, it is set to 3.")
            scroll_count = 3
        self.webscraper.check_category(category_url,scroll_count)
        self.log.info("Data extraction was done successfully.")

def run_scraper_single(self):
        seller_page_url = input("enter seller page url to crawl: ")
        self.get_driver()
        self.webscraper.check_seller(seller_page_url)
        self.log.info("Data extraction was done successfully.")

def run_seller_scraper_product(self):
        row_info = self.db_handler.get_row_info(fields=['seller_id', 'seller_name'], table_name='sellers',condition=None,return_as_list=False)
        if len(row_info) == 0 :
            self.log.warning("There are no sellers in the database.")
            return None
        
        for index, row in enumerate(row_info):
            self.log.info(f"ID {index} : {row[0]}, Name: {row[1]}")
        while True:
            try:
                user_pick = int(input('choose the seller ID you want to crawl products from: '))
                if 0 <= user_pick < len(row_info):
                    break
                else:
                    self.log.info('Invalid choose, please try again.')
            except ValueError:
                self.log.info('Invalid input, please enter a number.')
        selected_seller = row_info[user_pick]
        self.get_driver()
        self.log.info(f'You chose {selected_seller[1]} with ID {selected_seller[0]}')    
        seller_products = self.db_handler.get_row_info(['product_link', 'product_price'], 'products', ['seller_name', selected_seller[1]])
        available_products = [product[0] for product in seller_products if product[1] != 'product unavailable']
        self.log.info(f'{len(available_products)} available products found for {selected_seller[1]}')
        for index, link in enumerate(available_products):
            self.log.info(f'Starting to crawl - {index + 1}/{len(available_products)}')
            self.product_extraction_scraper.run(link)

def run_single_scraper_product(self):
        while True:
            url = input('Enter product URL: ')
            if 'product' not in url:
                self.log.info('URL must contain the word "product". Please try again.')
                continue
            if not re.search(r'dkp-\d+', url):
                self.log.info('URL must contain a product ID in the format "dkp-xxxxxx". Please try again.')
                continue
            break        
        self.log.info(f'Starting to crawl - {url.split("/")[4]}')
        self.get_driver()
        self.product_extraction_scraper.run(url)

def scraper_all_product_on_db(self,):
        seller_products = self.db_handler.get_row_info(['product_link', 'product_price'], 'products')
        available_products = [product[0] for product in seller_products if product[1] != 'product unavailable']
        self.log.info(f'{len(available_products)} available products found on database ')
        for index, link in enumerate(available_products):
            self.log.info(f'Starting to crawl - {index + 1}/{len(available_products)}')
            self.product_extraction_scraper.run(link)



In [ ]:
def unified_scraper(self, mode):
        url = None
        if mode in ['single_product', 'single_seller', 'category']:
            url = self.get_url_input(mode)
        if mode == 'category' : 
            scroll_count = input("Please enter the number of page scroll rates (Example 5 ) : ")
            try:
                scroll_count = int(scroll_count)
            except ValueError:
                self.log.error("The number of scrolling times must be a number. By default, it is set to 3.")
                scroll_count = 3
            
        # instalize driver 
            
        if mode == 'all_products':
            seller_products = self.db_handler.get_row_info(['product_link', 'product_price'], 'products')
            available_products = [product[0] for product in seller_products if product[1] != 'product unavailable']
            self.log.info(f'{len(available_products)} available products found on database ')
            for index, link in enumerate(available_products):
                self.log.info(f'Starting to crawl - {index + 1}/{len(available_products)}')
                self.product_extraction_scraper.run(link)
                
        elif mode == 'single_product':
            self.log.info(f'Starting to crawl - {url.split("/")[4]}')
            self.product_extraction_scraper.run(url)

        elif mode == 'seller_products':
            row_info = self.db_handler.get_row_info(fields=['seller_id', 'seller_name'], table_name='sellers',condition=None,return_as_list=False)
            if len(row_info) == 0 :
                self.log.warning("There are no sellers in the database.")
                return None
            
            for index, row in enumerate(row_info):
                self.log.info(f"ID {index} : {row[0]}, Name: {row[1]}")
            while True:
                try:
                    user_pick = int(input('choose the seller ID you want to crawl products from: '))
                    if 0 <= user_pick < len(row_info):
                        break
                    else:
                        self.log.info('Invalid choose, please try again.')
                except ValueError:
                    self.log.info('Invalid input, please enter a number.')
            
            selected_seller = row_info[user_pick]
            self.log.info(f'You chose {selected_seller[1]} with ID {selected_seller[0]}')    
            seller_products = self.db_handler.get_row_info(['product_link', 'product_price'], 'products', ['seller_name', selected_seller[1]])
            available_products = [product[0] for product in seller_products if product[1] != 'product unavailable']
            self.log.info(f'{len(available_products)} available products found for {selected_seller[1]}')
            for index, link in enumerate(available_products):
                self.log.info(f'Starting to crawl - {index + 1}/{len(available_products)}')
                self.product_extraction_scraper.run(link)

        elif mode == 'single_seller':
            self.webscraper.check_seller(url)
            self.log.info("Data extraction was done successfully.")
            
        elif mode == 'category':
            self.webscraper.check_category(url,scroll_count)
            self.log.info("Data extraction was done successfully.")  
            
        else:
            raise ValueError("Invalid mode selected")